<a href="https://colab.research.google.com/github/toppac/ACbook/blob/master/NovelAILeaks_API_Backend_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基于 NovelAILeaks 制作。

Credit: https://t.me/StableDiffusion_CN https://t.me/exlolicon

Thanks: 炼铜术士, 神楽坂早苗️, Jonathan, 咕 咕, 猫又逆变器, Gaein nidb

注意：该 notebook 会耗尽几乎所有 Colab GPU 资源，不建议频繁使用。



1. 下载 Novel AI API 后端、模型 （如果下载速度太慢可尝试 restart）

In [ ]:
%cd /content
!apt install -y -qq zstd
!curl -L https://cloudflare-ipfs.com/ipfs/bafybeiagafvge2siezdmkhiwrlj7obtjy6i3x7rbbaovw3247uk6xcob2e/sd-private.tar.zst | zstdcat | tar xf -
!curl -L https://cloudflare-ipfs.com/ipfs/bafybeidcibcsoso3ez453fnajqj7x5forntk6rkglvpp33h5pzsopdvwgq/modules.tar.zst | zstdcat | tar xf -
!curl -LO https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt 
!curl -L https://cloudflare-ipfs.com/ipfs/bafybeibhlcqjgvjtc3ck2bcazae5yizaetm7y7nm5rux674gsxgd4l5uce/animefull-latest.tar.zst | zstdcat | tar xf -


/content
zstd is already the newest version (1.3.3+dfsg-2ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  270M  100  270M    0     0   106M      0  0:00:02  0:00:02 --:--:--  106M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  920M  100  920M    0     0  98.6M      0  0:00:09  0:00:09 --:--:--  104M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  784M  100  784M    0     0   106M      0  0:00:07  0:00:07 --:--:--  109M
  % Total    % Receiv

2. 安装相关 package

In [ ]:
%cd /content
!pip install -U pip
!pip install dotmap icecream sentry-sdk numpy fastapi "uvicorn[standard]" gunicorn omegaconf transformers einops \
    pytorch_lightning simplejpeg min-dalle faiss-cpu sentence_transformers
!pip install -e "git+https://github.com/CompVis/taming-transformers.git#egg=taming-transformers" 

3. 安装 Novel AI 私有 package （参阅注释，约耗时30分钟）

In [ ]:
%cd /content
!pip install -e /content/sd-private/hydra-node-http/stable-diffusion-private/
!pip install https://github.com/NovelAI/k-diffusion-multigen/archive/457560c53c344b16cf3dda5eb0ba44d83e0b7c46.zip 
!pip install https://cloudflare-ipfs.com/ipfs/bafybeie5l2n7lyohudu2cu77uiv3pwjx6pwokaqszte7eg6qaz2hndfeiu/basedformer.tar 
!wget https://cloudflare-ipfs.com/ipfs/bafybeihrsvfo3oaldpcixzr5ds7rpu3gxguqhpifgbo53snborvdxyyb2i/xformers.zip 
!unzip xformers.zip

# 请根据 GPU 型号调整 TORCH_CUDA_ARCH_LIST，对应关系见
# https://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/
!cd xformers && env TORCH_CUDA_ARCH_LIST=7.5 python3 setup.py develop
!pip install -U --pre triton

4. 安装 bore 反向代理

In [ ]:
%cd /content
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin

5. 魔改一下

In [ ]:
%cd /content
!curl https://cloudflare-ipfs.com/ipfs/bafybeihqkt6hv454cry6lznz7hgn77zrjytibliqzwshlawvpkxxce2miq/hydra.tar | tar x -C /content/sd-private/hydra-node-http/hydra_node

# python3.7 backport
!python -m pip install git+https://github.com/pybpc/walrus
!python -m walrus /content/sd-private/hydra-node-http

6.  配置一个简单的前端，注意记录输出的映射地址

In [ ]:
%cd /content
!git clone https://github.com/nidbCN/novelai-sample

%cd /content/novelai-sample
!curl -Lo vue.config.js https://cloudflare-ipfs.com/ipfs/bafybeiaendc4ugufj6t6sgvoht27h3p46e4go4azn5igzds5l2xzjbl7g4/vue.config.js
!npm i
!nohup npm run serve &
!nohup bore local 8081 --to bore.pub &
!sleep 2
!tail nohup.out

7. 启动官方后端并使用 bore.pub 映射出来，注意记录输出的映射地址

In [ ]:
%cd /content/sd-private/hydra-node-http/

%env DTYPE=float16
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=/content/animefull-latest
%env MODULE_PATH=/content/modules/modules
%env TRANSFORMERS_CACHE=/content/transformer_cache
%env SENTRY_URL=
%env ENABLE_EMA=1
%env VAE_PATH=/content/animevae.pt
%env BASEDFORMER=1
%env PENULTIMATE=1

# start cf-argo-tunnel (optional)
# !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x cloudflared-linux-amd64
# !nohup ./cloudflared-linux-amd64 tunnel --url localhost:1830 &
# !sleep 10 && cat nohup.out

!gunicorn main:app --workers 1 --worker-class uvicorn.workers.UvicornWorker --bind 0.0.0.0:80 --timeout 0 --keep-alive 60 --log-level=debug & bore local 80 --to bore.pub

8. 使用方法

* 如果前端能够正常使用：使用前端 
* 如果本地有部署好的前端：使用本地前端 (github.com/nidbCN/novelai-sample)
* 如果前端不能正常使用：编写如下的脚本文件并运行，endpoint 填入第八步的 bore.pub:xxxx 地址

```python
import requests
import json
import base64
import random

endpoint = "http://bore.pub:xxxxx/generate"

data = {
  "prompt": "masterpiece, best quality, brown red hair,blue eyes,twin tails,holding cat", 
  "seed": random.randint(0, 2**32),
  "n_samples":1,
  "sampler":"ddim",
  "width":512,
  "height":768,
  "scale":11,
  "steps":28,
  "uc":"lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
}

req = requests.post(endpoint, json=data).json()
output = req["output"]
for x in output:
  img = base64.b64decode(x)
  with open("output-" + str(output.index(x)) + ".png", "wb") as f:
    f.write(img)
```

prompt中masterpiece, best quality, 开头对应原版web Add Quality Tags选项，不建议删除，后面直接跟自己prompt即可

uc部分对应web Undesired Content，建议保留默认

sampler是采样方法，可选plms/ddim/k_euler/k_euler_ancestral/k_heun/k_dpm_2/k_dpm_2_ancestral/k_lms

seed是种子，自己随机一个整数数字，不然一直会出一样的结果。

n_samples代表要生成几张图片